In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

from asl_bloch_sim import bloch, rf_design

In [ ]:
duration = 2 # seconds
dt = 0.0001 # seconds
B0 = 3 # Tesla

flip_angle = 180 # degrees
rf_duration = 0.012 # seconds
rf_bandwidth = 30000 # 250 # Hz
rf_stretch = 60
off_resonance = 400 # Hz
spectrum_lines = 400

T1 = 1.5 # seconds
T2 = 0.1 # seconds

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse_am, rf_pulse_fm = rf_design.adiabatic_pulse(flip_angle, rf_duration,
                                                     rf_bandwidth, rf_stretch, dt)


In [ ]:
rf_am = rf_design.extend(rf_pulse_am, duration, dt)
rf_fm = rf_design.extend(rf_pulse_fm, duration, dt)

dfz = np.linspace(0, off_resonance, spectrum_lines)
B = bloch.construct_B_field(rf_am, rf_fm, dfz)

In [ ]:
k = rf_design.adiabaticity(rf_pulse_am, rf_pulse_fm, B0, dt)

In [ ]:
k.min()

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')

plot.line(rf_time * 1e3, rf_pulse_am, line_width=2)
plot.yaxis.axis_label = 'RF Amplitude (µT)'
plot.y_range = bkm.Range1d(start=0, end=1.4e-4)
plot.extra_y_ranges['freq'] = bkm.Range1d(start=-rf_bandwidth - 1e3, end=rf_bandwidth + 1e3)
naxis = bkm.LinearAxis(y_range_name='freq', axis_label='Frequency (Hz)')
plot.add_layout(naxis, 'right')
plot.line(rf_time * 1e3, rf_pulse_fm, line_width=2, y_range_name='freq', color='green')

plot.xaxis.axis_label = 'Time (ms)'

bkp.show(plot)

In [ ]:
mag = np.array([[0, 0, 1]]) # initial magnetization
mags = np.array([mag := bloch.relax(bloch.precess(mag, B[step], dt), T1, T2, dt) for step in range(round(duration / dt))])

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(time, B[:, 0, 0], line_width=2, legend_label='Bx', alpha=0.5)
plot.line(time, B[:, 0, 1], line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(time, B[:, 0, 2], line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=0.14)
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(time, mags[:, 0, 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, mags[:, 0, 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, mags[:, 0, 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=0.14)
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[1], end // 10):
    alpha = 1 - offres / end
    plot.line(time, mags[:, offres, 2], line_width=2, legend_label=f'{dfz[offres]:g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0.01, end=0.05)
bkp.show(plot)

In [ ]:
flipped = np.argmin(mags[:, 0, 2], axis=0)
plot = bkp.figure(width=800, height=400, title='Flipped Magnetization Spectrum')
plot.line(dfz, mags[flipped, :, 2], line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
bkp.show(plot)